<a href="https://colab.research.google.com/github/RodasRios/DataCleaningAPUsGobDelValle/blob/main/DataAPUS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

!pip install ghostscript
!pip install camelot-py[base]
!pip install excalibur-py
!apt install ghostscript python3-tk
!pip install 'PyPDF2<3.0'

import PyPDF2

print("PyPDF2==" + PyPDF2.__version__)

import ghostscript
import camelot

!pip install pdfplumber

import pdfplumber
import re


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 706.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of camelot-py[base] to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.0/426.0 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

def extract_apu_from_pdf(pdf_path, max_pages=623):
    apus = []
    apu_pattern = re.compile(r'\b\d{2}-\d{2}-\d{2}\b')
    value_pattern = re.compile(r'^\d+([.,]\d+)*$')
    footer_pattern = re.compile(r'Gobernación del Valle del Cauca.*vivienda@valledelcauca.gov.co')

    current_apu = None
    apu_data = None
    collecting = False

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages[:max_pages]):
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                lines = [line for line in lines if not footer_pattern.search(line)]
                for j, line in enumerate(lines):
                    if apu_pattern.match(line.strip()):
                        if current_apu:
                            apus.append(apu_data)
                        current_apu = line.strip()
                        apu_data = {
                            'APU': current_apu,
                            'ITEM - Descripción': [],
                            'Unidad': [],
                            'Cant': [],
                            'Desper': [],
                            'Vr/Unitario': [],
                            'Vr/Parcial': []
                        }
                        collecting = True
                    elif current_apu and collecting:
                        parts = line.split()
                        if len(parts) >= 5 and value_pattern.match(parts[-1]):
                            apu_data['ITEM - Descripción'].append(" ".join(parts[:-5]))
                            apu_data['Unidad'].append(parts[-5])
                            apu_data['Cant'].append(parts[-4])
                            apu_data['Desper'].append(parts[-3])
                            apu_data['Vr/Unitario'].append(parts[-2])
                            apu_data['Vr/Parcial'].append(parts[-1])
                        elif len(parts) < 5 and value_pattern.match(parts[-1]):
                            collecting = False  # End of current APU if fewer parts found

    if current_apu:
        apus.append(apu_data)

    return apus

def apus_to_dataframes(apus):
    dataframes = []
    for apu in apus:
        data = {
            'ITEM - Descripción': apu['ITEM - Descripción'],
            'Unidad': apu['Unidad'],
            'Cant': apu['Cant'],
            'Desper': apu['Desper'],
            'Vr/Unitario': apu['Vr/Unitario'],
            'Vr/Parcial': apu['Vr/Parcial']
        }
        df = pd.DataFrame(data)
        if df.empty:
            continue
        dataframes.append(df)
    return dataframes

# Ruta al archivo PDF
pdf_path = '/content/APU.pdf'

# Extraer los APU de las primeras 2 páginas
apus = extract_apu_from_pdf(pdf_path, max_pages=623)

# Convertir los APU extraídos a DataFrames
apu_dataframes = apus_to_dataframes(apus)

# Imprimir el DataFrame del primer APU
if apu_dataframes:
    print(apu_dataframes[0])
else:
    print("No se encontraron APU en las primeras páginas.")




                        ITEM - Descripción Unidad   Cant Desper Vr/Unitario  \
0  MOAG01-MANO OBRA ALBANILERIA 1 AYUDANTE     HC  6,000   0,00       8.949   
1                     MQ0207-VOLQUETA 5 M3    VJE  1,000   0,00      46.500   
2                 MQ0301-HERRAMIENTA MENOR    GLB  4,000   0,00       1.600   

  Vr/Parcial  
0     53.694  
1     46.500  
2      6.400  


In [ ]:
import pdfplumber
import re

def preprocess_pdf(pdf_path, output_path, max_pages=623):
    merged_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            if max_pages is not None and i >= max_pages:
                break
            text = page.extract_text()
            # Eliminar encabezado
            header_index = text.find("Encabezado a eliminar")
            if header_index != -1:
                text = text[header_index + len("Encabezado a eliminar"):]
            # Eliminar pie de página
            footer_index = text.find("Gobernación del Valle del Cauca - Carrera 6 Calle 9 y 10 Teléfonos:")
            if footer_index != -1:
                text = text[:footer_index]
            merged_text += text + "\n"

    # Eliminar espacios adicionales entre líneas
    merged_text = re.sub(r'\n\s*\n', '\n', merged_text, flags=re.M)

    # Guardar el texto en un archivo de texto
    with open(output_path, 'w') as f:
        f.write(merged_text)

    # Leer el archivo para eliminar líneas específicas
    with open(output_path, 'r') as f:
        lines = f.readlines()

    # Eliminar las líneas específicas
    lines_to_remove = [33488, 33489]  # Líneas a eliminar
    lines = [line for i, line in enumerate(lines) if i not in lines_to_remove]

    # Guardar el texto modificado en el mismo archivo
    with open(output_path, 'w') as f:
        f.writelines(lines)

# Ruta al archivo PDF
pdf_path = '/content/APU.pdf'
# Ruta donde guardar el texto procesado
output_path = 'archivo_procesado.txt'

# Procesar el PDF y guardar el texto en un archivo
preprocess_pdf(pdf_path, output_path)



In [ ]:
def extract_totales_apus_from_text(text_path):
    totales_apus = []
    keywords = ['Materiales', 'Equipo', 'AIU', 'SubTotal', 'Mano de Obra', 'Otros', 'VALOR TOTAL']
    keyword_pattern = re.compile(r'\bMateriales\b.*\bEquipo\b.*\bAIU\b.*\bSubTotal\b.*\bMano de Obra\b.*\bOtros\b.*\bVALOR TOTAL\b')

    with open(text_path, 'r') as f:
        text = f.read()

    lines = text.split('\n')
    for i, line in enumerate(lines):
        if keyword_pattern.match(line.strip()):
            if i + 1 < len(lines):
                parts = lines[i + 1].split()
                if len(parts) == len(keywords):
                    totales_apus.append(parts)

    if totales_apus:
        totales_apus_df = pd.DataFrame(totales_apus, columns=keywords)
        return totales_apus_df
    else:
        return pd.DataFrame(columns=keywords)

# Ruta al archivo de texto procesado
text_path = '/content/archivo_procesado.txt'

# Extraer los totales de los APUs del archivo de texto procesado
totales_apus_df = extract_totales_apus_from_text(text_path)

# Mostrar el DataFrame con los totales de los APUs
print(totales_apus_df[33:35])

   Materiales Equipo AIU SubTotal Mano de Obra Otros VALOR TOTAL
33     51.626    480   0   52.106       13.800     0     $65.906
34     74.378    480   0   74.858       17.084     0     $91.942


In [ ]:
def extract_apu_labels_from_pdf(pdf_path, max_pages=623):
    apu_labels = []
    apu_pattern = re.compile(r'\b\d{2}-\d{2}-\d{2}\b')

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages[:max_pages]):
            text = page.extract_text()
            if text:
                lines = text.split('\n')
                for line in lines:
                    match = apu_pattern.search(line)
                    if match:
                        codigo = match.group()
                        rest = line[match.end():].strip()
                        # Split the rest of the line by spaces, taking the last part as the unit
                        parts = rest.split()
                        unidad = parts[-1] if len(parts) > 1 else None
                        descripcion = ' '.join(parts[:-1]) if len(parts) > 1 else parts[0]
                        apu_labels.append([codigo, descripcion, unidad])


    if apu_labels:
        df_totales_apus = pd.DataFrame(apu_labels, columns=['Codigo', 'Descripcion', 'UnidadAPU'])
        return df_totales_apus
    else:
        return pd.DataFrame(columns=['Codigo', 'Descripcion', 'UnidadAPU'])



# Ruta al archivo PDF
pdf_path = '/content/APU.pdf'

# Extraer los códigos, descripciones y unidades de los APUs de las primeras 2 páginas
apu_labels_df = extract_apu_labels_from_pdf(pdf_path, max_pages=623)
# Eliminar el primer caracter de la columna Descripcion
apu_labels_df['Descripcion'] = apu_labels_df['Descripcion'].str.slice(1)

# Imprimir el DataFrame de códigos, descripciones y unidades de los APUs
#print(apu_labels_df)


In [ ]:
def complete_apus_with_totales(apu_dataframes, totales_apus_df, apu_labels_df):
    keywords = ['Materiales', 'Equipo', 'AIU', 'SubTotal', 'Mano de Obra', 'Otros', 'VALOR TOTAL']
    keywords_labels = ['Codigo', 'Descripcion', 'UnidadAPU']

    # Agregar las nuevas columnas con valores None a cada DataFrame
    for df in apu_dataframes:
        for keyword in keywords:
            df[keyword] = None

    # Agregar las nuevas columnas con valores None a cada DataFrame
    for df in apu_dataframes:
        for keywordL in keywords:
            df[keywords_labels] = None

    # Completar los APUs con los valores de totales
    for i, df in enumerate(apu_dataframes):
        if i + 1 < len(totales_apus_df):  # Empezar desde el índice 1
            for keyword in keywords:
                df.at[0, keyword] = totales_apus_df.at[i + 1, keyword]

    # Completar los APUs con las etiquetas
    for j, df in enumerate(apu_dataframes):
        if j < len(apu_labels_df):
            for keywordL in keywords_labels:
                df.at[0, keywordL] = apu_labels_df.at[j, keywordL]

    return apu_dataframes

# Completar los APUs con los valores de totales y etiquetas
APU_completados = complete_apus_with_totales(apu_dataframes, totales_apus_df, apu_labels_df)

# Imprimir el DataFrame del primer APU completado
#if APU_completados:
  #  print(APU_completados[0])
#else:
   # print("No se encontraron APU en las primeras páginas.")

In [ ]:
APU_completados[200]

,ITEM - Descripción,Unidad,Cant,Desper,Vr/Unitario,Vr/Parcial,Materiales,Equipo,AIU,SubTotal,Mano de Obra,Otros,VALOR TOTAL,Codigo,Descripcion,UnidadAPU
0,004980-VAL RETENC ASTM A-126 3EB,UND,"1,000","0,00",429.200,429.200,429.200,1.600,0,430.800,76.173,0,$506.973,04-05-67,VALV RETENCION ASTM A-126 3 EB,UND
1,MOIS01-MANO OBRA HIDROSANIT. 1 AYUDANTE-1 OFI,HC,"3,000","0,00",25.391,76.173,None,None,None,None,None,None,None,None,None,None
2,MQ0301-HERRAMIENTA MENOR,GLB,"1,000","0,00",1.600,1.600,None,None,None,None,None,None,None,None,None,None


In [ ]:
APU_completados[500].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ITEM - Descripción  6 non-null      object
 1   Unidad              6 non-null      object
 2   Cant                6 non-null      object
 3   Desper              6 non-null      object
 4   Vr/Unitario         6 non-null      object
 5   Vr/Parcial          6 non-null      object
 6   Materiales          1 non-null      object
 7   Equipo              1 non-null      object
 8   AIU                 1 non-null      object
 9   SubTotal            1 non-null      object
 10  Mano de Obra        1 non-null      object
 11  Otros               1 non-null      object
 12  VALOR TOTAL         1 non-null      object
 13  Codigo              1 non-null      object
 14  Descripcion         1 non-null      object
 15  UnidadAPU           1 non-null      object
dtypes: object(16)
memory usage: 89

In [ ]:
###for i, df in enumerate(APU_completados):
###    df.to_csv(f'APU_completado_{i}.csv', index=False)

In [ ]:
def process_apu_data(df_list):
    for df in df_list:

      df['Desper'] = df['Desper'].str.replace(',', '.').astype(float)
      df['Cant'] = df['Cant'].str.replace(',', '.').astype(float)
      df['Vr/Unitario'] = df['Vr/Unitario'].str.replace('.', '').astype(float)
      df['Vr/Parcial'] = df['Vr/Parcial'].str.replace('.', '').astype(float)


      df['VALOR TOTAL'] = df['VALOR TOTAL'].str.replace('.', '').str.replace('$', '')
      df['Equipo'] = df['Equipo'].str.replace('.', '')
      df['AIU'] = df['AIU'].str.replace('.', '')
      df['SubTotal'] = df['SubTotal'].str.replace('.', '')
      df['Materiales'] = df['Materiales'].str.replace('.', '')
      df['Otros'] = df['Otros'].str.replace('.', '')
      df['Mano de Obra'] = df['Mano de Obra'].str.replace('.', '')


      df['VALOR TOTAL'] = df['VALOR TOTAL'].apply(lambda x: float(x) if x is not None else None)
      df['Equipo'] = df['Equipo'].apply(lambda x: float(x) if x is not None else None)
      df['AIU'] = df['AIU'].apply(lambda x: float(x) if x is not None else None)
      df['SubTotal'] = df['SubTotal'].apply(lambda x: float(x) if x is not None else None)
      df['Materiales'] = df['Materiales'].apply(lambda x: float(x) if x is not None else None)
      df['Mano de Obra'] = df['Mano de Obra'].apply(lambda x: float(x) if x is not None else None)
      df['Otros'] = df['Otros'].apply(lambda x: float(x) if x is not None else None)


    return df_list

# Procesar los datos en APU_completados
APU_completados_procesados = process_apu_data(APU_completados)

In [ ]:
APU_completados_procesados[200]

,ITEM - Descripción,Unidad,Cant,Desper,Vr/Unitario,Vr/Parcial,Materiales,Equipo,AIU,SubTotal,Mano de Obra,Otros,VALOR TOTAL,Codigo,Descripcion,UnidadAPU
0,004980-VAL RETENC ASTM A-126 3EB,UND,1.0,0.0,429200.0,429200.0,429200.0,1600.0,0.0,430800.0,76173.0,0.0,506973.0,04-05-67,VALV RETENCION ASTM A-126 3 EB,UND
1,MOIS01-MANO OBRA HIDROSANIT. 1 AYUDANTE-1 OFI,HC,3.0,0.0,25391.0,76173.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2,MQ0301-HERRAMIENTA MENOR,GLB,1.0,0.0,1600.0,1600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None


In [ ]:
for df in APU_completados_procesados:
    df.insert(13, 'VALOR TOTAL 2024', df['VALOR TOTAL'].apply(
        lambda x: round(x * (1 + 0.0562) * (1 + 0.1312) * (1 + 0.0928), 0) if pd.notna(x) else None
    ))

for df in APU_completados_procesados:
    df.insert(6, 'Vr/Parcial 2024', df['Vr/Parcial'].apply(
        lambda x: round(x * (1 + 0.0562) * (1 + 0.1312) * (1 + 0.0928), 0) if pd.notna(x) else None
    ))










In [ ]:
APU_completados_procesados[500]

,ITEM - Descripción,Unidad,Cant,Desper,Vr/Unitario,Vr/Parcial,Vr/Parcial 2024,Materiales,Equipo,AIU,SubTotal,Mano de Obra,Otros,VALOR TOTAL,VALOR TOTAL 2024,Codigo,Descripcion,UnidadAPU
0,004465-CONECTOR TIPO CUNA,UND,1.0,0.0,18000.0,18000.0,23502.0,372832.0,6400.0,0.0,379232.0,63035.0,0.0,442267.0,577445.0,06-10-54,T1 CON D.I. TIERRA CON DUCTO INTERNO,UND
1,004486-ALAMBRE CUBRE D.D. # 4,ML,10.0,0.0,16680.0,166800.0,217782.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2,004570-VARILLA COBRE,UND,1.0,0.0,182000.0,182000.0,237628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
3,005803-CONECTOR VARILLA COBRE,UND,1.0,0.0,6032.0,6032.0,7876.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
4,MOIE03-MANO OBRA ING.ELECTR 3 AYUD-1 OFI-1 PROF,HC,0.6,0.0,105059.0,63035.0,82302.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
5,MQ0301-HERRAMIENTA MENOR,GLB,4.0,0.0,1600.0,6400.0,8356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None


In [ ]:
cantidad_apus = len(APU_completados_procesados)
print(f"La cantidad de APU es: {cantidad_apus}")

La cantidad de APU es: 3682


In [ ]:
# Crear un nuevo DataFrame con las columnas necesarias
Precios = pd.DataFrame(columns=['Codigo', 'Descripcion', 'UnidadAPU', 'Precio Total'])

# Iterar sobre los DataFrames completados
for df in APU_completados:
    # Seleccionar las columnas necesarias
    subset = df[['Codigo', 'Descripcion', 'UnidadAPU', 'VALOR TOTAL']]
    # Renombrar la columna VALOR TOTAL a Precio Total
    subset = subset.rename(columns={'VALOR TOTAL': 'Precio Total'})
    # Agregar las filas al DataFrame Precios
    Precios = pd.concat([Precios, subset], ignore_index=True)
# Eliminar filas con valores nulos en el DataFrame Precios
Precios = Precios.dropna()
# Mostrar el DataFrame Precios sin filas nulas
#print(Precios)

# Exportar el DataFrame Precios a un archivo CSV llamado 'Precios.csv'
Precios.to_csv('Precios.csv', index=False)

In [ ]:
# Suponiendo que APU_completados es una lista de DataFrames de Pandas
# Creamos un diccionario donde cada clave es un índice único y cada valor es un DataFrame
dataframes_dict = {f"df_{idx}": df for idx, df in enumerate(APU_completados)}

# Guardamos el diccionario en un archivo JSON
with open('data.json', 'w') as file:
    for key, value in dataframes_dict.items():
        file.write('{"' + key + '":' + value.to_json(orient='records') + '}\n')

# Cargamos el archivo JSON y reconstruimos el diccionario de DataFrames
loaded_dataframes_dict = {}
with open('data.json', 'r') as file:
    for line in file:
        data = pd.read_json(line.strip(), orient='records')
        key = list(data.keys())[0]
        loaded_dataframes_dict[key] = data[key]

# Ahora loaded_dataframes_dict contiene el diccionario de DataFrames cargado desde el archivo JSON

In [ ]:
# Cargar el archivo JSON y reconstruir la lista de DataFrames originales
APUS_prueba = []
with open('data.json', 'r') as file:
    for line in file:
        # Cargar el diccionario de DataFrames desde el archivo JSON
        data = pd.read_json(line.strip(), typ='series').to_dict()
        for key, value in data.items():
            # Reconstruir cada DataFrame con su nombre original
            idx = int(key.split('_')[1])  # Obtener el índice del DataFrame del nombre de la clave
            if len(APUS_prueba) <= idx:  # Asegurarse de que haya suficientes elementos en la lista
                APUS_prueba.extend([pd.DataFrame()] * (idx - len(APUS_prueba) + 1))
            APUS_prueba[idx] = pd.DataFrame(value)

# APU_completados ahora contiene la lista de DataFrames originales

In [ ]:
APUS_prueba[2500]

,ITEM - Descripción,Unidad,Cant,Desper,Vr/Unitario,Vr/Parcial,Vr/Parcial 2024,Materiales,Equipo,AIU,SubTotal,Mano de Obra,Otros,VALOR TOTAL,VALOR TOTAL 2024,Codigo,Descripcion,UnidadAPU
0,"000306-""BISAGRA 3x1.1/4"""" COBRI",PAR,3.0,0.0,7651.0,22953.0,29969.0,199500.0,8000.0,0.0,207500.0,182222.0,0.0,389722.0,508840.0,21-01-18,NAVE MAD.TRIPLEX 81-100 (1N-B),UND
1,001495-LAM.TRIPLEX 3.2MM,UND,2.0,0.0,28900.0,57800.0,75466.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2,002036-PEGANTE A X W,GLN,0.2,0.0,65900.0,13180.0,17208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
3,002121-LISTON 2 x3x300 AMARIL,UND,2.0,5.0,18000.0,37800.0,49354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
4,003500-PUNTILLA 1 SC,LBS,0.5,0.0,3900.0,1950.0,2546.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
5,003873-MAD.CEDRO CAQUETA,PG2,9.0,0.0,7313.0,65817.0,85934.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
6,MOCO01-MANO OBRA CARP.MADERA 1 AYUDANTE-1 OFI,HC,3.0,0.0,26886.0,80658.0,105311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
7,MOCT01-MANO OBRA CARP.TALLER 1 AYUDANTE-1 OFI,HC,4.0,0.0,25391.0,101564.0,132607.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
8,MQ0301-HERRAMIENTA MENOR,GLB,5.0,0.0,1600.0,8000.0,10445.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None


In [ ]:
APU_completados[30]

,ITEM - Descripción,Unidad,Cant,Desper,Vr/Unitario,Vr/Parcial,Materiales,Equipo,AIU,SubTotal,Mano de Obra,Otros,VALOR TOTAL,Codigo,Descripcion,UnidadAPU
0,000239-BALASTRO DE RIO,M3,0.05,5.0,19500.0,1024.0,16824.0,480.0,0.0,17304.0,7231.0,0.0,24534.0,02-03-04,"TUB CONCRETO SIMPLE D= 6""UNION CAUCHO",ML
1,"002959-""TUBO CONCRETO CII 6""""",ML,1.00,2.0,15490.0,15800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
2,MOAG12-MANO OBRA ALBANILERIA 2 AYUDANTE-1 OFI,HC,0.20,0.0,32845.0,6569.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
3,MOTO01-MANO OBRA TOPOGRAFIA 1 CADENERO-1 TOP,HC,0.01,0.0,66168.0,662.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
4,MQ0301-HERRAMIENTA MENOR,GLB,0.30,0.0,1600.0,480.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
